In [1]:
import os
os.chdir('../scripts/')
from sentinel2 import FarmKML

In [2]:
votm = FarmKML()

In [3]:
votm_m = votm.grazeable_gdf.to_crs(crs='epsg:32616')

In [4]:
votm_m[votm_m.Name == 'buffered'].geometry.area

26    473992.425107
dtype: float64

In [4]:
votm.grazeable_gdf

,Name,Description,geometry
0,Paddock 1,,"POLYGON Z ((-89.14747 42.17183 0.00000, -89.14..."
1,Paddock 2,,"POLYGON Z ((-89.14617 42.17203 0.00000, -89.14..."
2,Paddock 3,,"POLYGON Z ((-89.14696 42.17160 0.00000, -89.14..."
3,Paddock 4,,"POLYGON Z ((-89.14654 42.17100 0.00000, -89.14..."
4,Paddock 5,,"POLYGON Z ((-89.14800 42.17076 0.00000, -89.14..."
5,Paddock 6,,"POLYGON Z ((-89.14650 42.17032 0.00000, -89.14..."
6,Paddock 7,,"POLYGON Z ((-89.14660 42.17030 0.00000, -89.14..."
7,Paddock 8,,"POLYGON Z ((-89.14644 42.16978 0.00000, -89.14..."
8,Paddock 9,,"POLYGON Z ((-89.14787 42.17019 0.00000, -89.14..."
9,Paddock 10,,"POLYGON Z ((-89.14643 42.16973 0.00000, -89.14..."
